In [2]:
!conda install -q pytorch torchvision cudatoolkit=11 -c pytorch-nightly

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with repodata from current_repodata.json, will retry with next repodata source.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... 
Found conflicts! Looking for incompatible packages.
This can take several minutes.  Press CTRL-C to abort.
failed

Examining @/win-64::__cuda==11.3=0:  86%|████████▌ | 6/7 [00:01<00:00,  4.21it/s]
                                                                                 

Examining conflict for pytorch torchvision:   0%|          | 0/7 [00:00<?, ?it/s]
Examining conflict for pytorch python torchvision:  14%|█▍        | 1/7 [00:00<00:00, 10.26it/s]
Examining conflict for pytorch python cudatoolkit torchvision:  29%|██▊       | 2/7 [00:00<00:00, 10.34it/s]
Examining conflict for pytorch python cudatoolkit torchvision:  43%|████▎   

In [5]:
import pandas as pd
import numpy as np 
from torch import nn
import torch
from torchtext import data
from torch.nn  import functional as F
import torch.optim as  optim 
df = pd.read_csv("data/arabic_english.txt",delimiter="\t",names=["eng","ar"])
df

,eng,ar
0,Hi.,مرحبًا.
1,Run!,اركض!
2,Help!,النجدة!
3,Jump!,اقفز!
4,Stop!,قف!
...,...,...
24633,rising voices promoting a more linguistically ...,شاركنا تحدي ابداع ميم بلغتك الام تعزيزا للتنوع...
24634,following last year s successful campaign we i...,استكمالا لنجاح حملة العام السابق ندعوكم للمشار...
24635,during last year s challenge we also met langu...,تعرفنا خلال تحدي العام الماضي على ابطال لغويين...
24636,to take part just follow the simple steps outl...,للمشاركة في التحدي اتبع الخطوات الموضحة على ال...


In [4]:
!python -m spacy download en_core_web_sm


Python est introuvable. Ex�cutez sans argument pour proc�der � l


# Tokenizer

In [10]:
import math
import pandas as pd
import math
import pandas as pd
import torchtext
import torch
import torch.nn as nn
import random
import re
import spacy
from torchtext import data
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
from spacy.lang.ar import Arabic
from nltk.translate.bleu_score import sentence_bleu
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import Vocab
from torch import Tensor


random.seed(0)
df = pd.read_csv("data/arabic_english.txt",delimiter="\t",names=["eng","ar"])

'''
First :
python -m spacy download en_core_web_sm
'''
spacy_eng = spacy.load("en_core_web_sm")

ar = Arabic()
ar_Tokenizer = Tokenizer(ar.vocab)

def engTokenizer(text):
 return  [word.text for word in spacy_eng.tokenizer(text)] 

def arTokenizer(sentence):
    return  [word.text for word in 
             ar_Tokenizer(re.sub(r"\s+"," ",re.sub(r"[\.\'\"\n+]"," ",sentence)).strip())]

SRC = data.Field(tokenize=engTokenizer,batch_first=False,init_token="<sos>",eos_token="<eos>")
TARGET = data.Field(tokenize=arTokenizer,batch_first=False,tokenizer_language="ar",init_token="ببدأ",eos_token="نهها")

class TextDataset(data.Dataset):

    def __init__(self, df, src_field, target_field, is_test=False, **kwargs):
        fields = [('eng', src_field), ('ar',target_field)]
        samples = []
        for i, row in df.iterrows():
            eng = row.eng 
            ar = row.ar
            samples.append(data.Example.fromlist([eng, ar], fields))

        super().__init__(samples, fields, **kwargs)
    
    def __len__(self):
        return len(self.samples)

    def __getitem__(self, idx):
        return self.samples[idx]

torchdataset = TextDataset(df,SRC,TARGET)

train_data, valid_data = torchdataset.split(split_ratio=0.8, random_state = random.seed(0))

SRC.build_vocab(train_data,min_freq=2)
TARGET.build_vocab(train_data,min_freq=2)

print(TARGET.vocab.freqs.most_common(50))  




[('في', 10114), ('من', 8806), ('على', 5213), ('ان', 2562), ('عن', 2299), ('العالمية', 2260), ('الاصوات', 2191), ('الى', 1867), ('لا', 1599), ('هذا', 1572), ('ما', 1385), ('التي', 1384), ('هذه', 1157), ('مع', 1077), ('الذي', 847), ('أن', 824), ('ذلك', 808), ('كان', 799), ('لم', 797), ('او', 797), ('الانترنت', 785), ('توم', 733), ('هل', 715), ('و', 715), ('كل', 671), ('بعد', 668), ('هو', 616), ('قبل', 580), ('تم', 572), ('موقع', 562), ('حول', 552), ('عام', 505), ('العالم', 496), ('حيث', 488), ('كما', 485), ('بين', 463), ('اكثر', 448), ('المدون', 447), ('قد', 445), ('غير', 441), ('خلال', 432), ('أنا', 432), ('إلى', 428), ('يوم', 427), ('هناك', 426), ('كانت', 424), ('بعض', 420), ('ايضا', 419), ('هي', 419), ('اي', 418)]


In [11]:
class Transformer(nn.Module):
    def __init__(
        self,
        embedding_size,
        src_vocab_size,
        trg_vocab_size,
        src_pad_idx,
        num_heads,
        num_encoder_layers,
        num_decoder_layers,
        forward_expansion,
        max_len,
        device,
    ):
        super(Transformer, self).__init__()
        self.src_word_Embeddings = nn.Embedding(src_vocab_size,embedding_size)
        self.src_Positional_Embeddings= nn.Embedding(max_len,embedding_size)
        self.trg_Embeddings= nn.Embedding(trg_vocab_size,embedding_size)
        self.trg_Positional_Embeddings= nn.Embedding(max_len,embedding_size)
        self.device = device
        self.transformer = nn.Transformer(
            embedding_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )

        self.fc_out = nn.Linear(embedding_size, trg_vocab_size)
        self.dropout = nn.Dropout(0.1)
        self.src_pad_idx = src_pad_idx
    
    def make_src_mask(self, src):
        src_mask = src.transpose(0,1) == self.src_pad_idx

        return src_mask.to(device)

    def forward(self,src,trg):
        src_seq_length = src.shape
        trg_seq_length = trg.shape
        #adding zeros is an easy way
        src_positions = (
            torch.arange(0, src_seq_length)
            .reshape(src_seq_length,1)  + torch.zeros(src_seq_length,N) 
        ).to(device)
        
        trg_positions = (
            torch.arange(0, trg_seq_length)
            .reshape(trg_seq_length,1)  + torch.zeros(trg_seq_length,N) 
        ).to(device)


        embed_src = self.dropout(self.srcEmbeddings(x.long()) +self.srcPositionalEmbeddings(src_positions.long()))

        embed_trg = self.dropout(self.trgEmbeddings(trg.long())+self.trgPositionalEmbeddings(trg_positions.long()))
        
        src_padding_mask = self.make_src_mask(x)
        trg_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(device)
        
        
        out = self.transformer(srcWords,trgWords, src_key_padding_mask=src_padding_mask,tgt_mask=trg_mask )
        out= self.fc_out(out)
        return out
        

# Training phase

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_model = False
save_model = True

num_epochs = 5
learning_rate = 3e-4
BATCH_SIZE = 32

num_heads = 8
num_encoder_layers = 3
num_decoder_layers = 3

max_len= 250
dropout = 0.10
embedding_size= 512
src_pad_idx = SRC.vocab.stoi["<pad>"]
forward_exânsion = 4
step = 0



train_iterator,valid_iterator = data.BucketIterator.splits(
    (train_data,valid_data), 
    batch_size = BATCH_SIZE,
    sort=False,
    sort_within_batch=False,
    sort_key = lambda x: len(x.SRC),
    device=device
)



src_vocab_size  = len(SRC.vocab)
print("Size of english vocabulary:",src_vocab_size)

#No. of unique tokens in label
trg_vocab_size =len(TARGET.vocab)
print("Size of arabic vocabulary:",trg_vocab_size)


model = Transformer(        
    embedding_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_exânsion,
    max_len,
    device
).to(device)




Size of english vocabulary: 12886
Size of arabic vocabulary: 22062


TypeError: __init__() takes 11 positional arguments but 12 were given

In [1]:
loss_track = []
loss_validation_track= []


optimizer = optim.Adam(model.parameters(), lr=learning_rate)
pad_idx = english.vocab.stoi["<pad>"]
criterion = nn.CrossEntropyLoss(ignore_idx = pad_idx)

if load_model: load_checkpoint(torch.load("my_checkpoint.pth.ptar"),model, optimizer)

for epoch in range(NUM_EPOCHS):
    stepLoss=[]
    model.train() # the training mode for the model (applies dropout and batchnorms)
    for batch  in enumerate(train_iterator):
        input_sentence = batch.eng.to(device)
        trg = batch.ar.to(device)

        out = model(input_sentence,trg[:-1])


        out = out.reshape(-1,trg_vocab_size)
        trg = trg[1:].reshape(-1)
        optimizer.zero_grad()


        loss = criterion(out,trg)
        loss.backward()

        optimizer.step()
        
        stepLoss.append(loss.item())
        

    loss_track.append(np.mean(stepLoss))
    print("train crossentropy at epoch {} loss: ".format(i),np.mean(stepLoss))

NameError: name 'optim' is not defined

Size of english vocabulary: 12886
Size of arabic vocabulary: 22062


TypeError: __init__() missing 2 required positional arguments: 'max_len' and 'device'